In [1]:
import pandas as pd
import numpy as np
import io
import requests
import locale
import json
import os
import pymupdf
import concurrent.futures
import tiktoken
from collections import defaultdict
from base64 import b64encode
from datetime import datetime, timedelta
from requests.auth import HTTPBasicAuth

In [2]:
URL = f'https://analytics.dev.migtra.com/?company=%s&ra=%s'
basic = HTTPBasicAuth('reportes_semanales', 'qSE0Gg3rR4BL6nWRZKE1PSO')

In [3]:
def get_table(company: str, ra: str) -> pd.DataFrame:
    res = requests.get(URL % (company, ra), auth= basic, timeout = 10)
    csv_data = io.StringIO(res.text)
    df = pd.read_csv(csv_data)
    return df

In [4]:
company = 'abastible_consolidado'
ra = 'raev'
RIESGO = {
    "Bajo": 6.1,
    "Medio": 8.2,
    "Alto": 9.8
}

In [5]:
# data = get_table(company, ra)
data = pd.read_csv(f'Clientes/{company}/raev.csv')
data

,Oficina,Flota,Codigo,Date,bajo,medio,alto,distance,raev,dev
0,MAIPU,ENVASADO,EN4325,2025-01-27,4.9,6.2,8.2,0.000000,0.000000,0.000000
1,LENGA,GRANEL,GR5092,2025-01-27,4.9,6.2,8.2,270.853634,6.016551,4.460648
2,MAIPU,ENVASADO,EN2980,2025-01-27,4.9,6.2,8.2,0.000000,0.000000,0.000000
3,MAIPU,GRANEL,GR5054,2025-01-27,4.9,6.2,8.2,108.274714,15.757812,10.531762
4,MAIPU,GRANEL,GR5151,2025-01-27,4.9,6.2,8.2,134.626635,14.504033,9.938442
...,...,...,...,...,...,...,...,...,...,...
513367,LENGA,GRANEL,GR5109,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000
513368,OSORNO,GRANEL,GR5110,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000
513369,LENGA,TRANSFERENCIA,EN3219,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000
513370,LENGA,TRANSFERENCIA,EN3220,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000


In [6]:
df = data.copy()

In [7]:
df['Date'] = pd.to_datetime(df['Date'])
df

,Oficina,Flota,Codigo,Date,bajo,medio,alto,distance,raev,dev
0,MAIPU,ENVASADO,EN4325,2025-01-27,4.9,6.2,8.2,0.000000,0.000000,0.000000
1,LENGA,GRANEL,GR5092,2025-01-27,4.9,6.2,8.2,270.853634,6.016551,4.460648
2,MAIPU,ENVASADO,EN2980,2025-01-27,4.9,6.2,8.2,0.000000,0.000000,0.000000
3,MAIPU,GRANEL,GR5054,2025-01-27,4.9,6.2,8.2,108.274714,15.757812,10.531762
4,MAIPU,GRANEL,GR5151,2025-01-27,4.9,6.2,8.2,134.626635,14.504033,9.938442
...,...,...,...,...,...,...,...,...,...,...
513367,LENGA,GRANEL,GR5109,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000
513368,OSORNO,GRANEL,GR5110,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000
513369,LENGA,TRANSFERENCIA,EN3219,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000
513370,LENGA,TRANSFERENCIA,EN3220,2021-01-01,4.9,6.2,8.2,0.000000,0.000000,0.000000


In [8]:
df['Oficina'] = df['Oficina'].str.replace('DISPONIBLES', 'SANTIAGO')
df[df['Oficina'] == 'SANTIAGO']

,Oficina,Flota,Codigo,Date,bajo,medio,alto,distance,raev,dev
45,SANTIAGO,TRANSFERENCIA,EN6500,2025-01-27,4.9,6.2,8.2,0.000000,0.000000,0.000000
93,SANTIAGO,TRANSFERENCIA,EN5975,2025-01-27,4.9,6.2,8.2,271.011367,47.565276,32.596107
477,SANTIAGO,TRANSFERENCIA,EN6500,2025-01-26,4.9,6.2,8.2,0.000000,0.000000,0.000000
521,SANTIAGO,TRANSFERENCIA,EN5975,2025-01-26,4.9,6.2,8.2,0.000000,0.000000,0.000000
797,SANTIAGO,TRANSFERENCIA,EN6500,2025-01-25,4.9,6.2,8.2,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
29458,SANTIAGO,TRANSFERENCIA,EN6333,2024-11-02,4.9,6.2,8.2,621.890720,4.037598,2.660868
29462,SANTIAGO,TRANSFERENCIA,EN5975,2024-11-02,4.9,6.2,8.2,270.163145,72.850039,54.782099
29742,SANTIAGO,TRANSFERENCIA,EN6500,2024-11-01,4.9,6.2,8.2,0.000000,0.000000,0.000000
29749,SANTIAGO,TRANSFERENCIA,EN6333,2024-11-01,4.9,6.2,8.2,0.000000,0.000000,0.000000


# Funciones

In [9]:
def capitalizePalabras(s: str) -> str:
    """Capitalizar las palabras de un string
    """
    return ' '.join([word.capitalize() for word in s.split()])

In [10]:
locale.setlocale(locale.LC_TIME, 'es_CL.UTF-8')

def getRAEV100(df):
    df['raev100'] = ((df['raevSum'] / df['distanceSum']) * 100).round(1)
    return df

In [37]:
def getRAEV100Level(df: pd.DataFrame, bajo: float, medio: float, alto: float) -> pd.DataFrame:
    """Obtener el nivel de RAEV 100 de acuerdo a los umbrales
    establecidos.
    """
    df['Riesgo'] = df['raev100'].apply(lambda x: 'Bajo' if x < bajo else 'Medio' if x < medio else 'Alto' if x < alto else 'Muy Alto' if pd.isna(x) else 'N/A')
    return df

# JSON

In [60]:
def buildJSON(df: pd.DataFrame, company: str) -> dict:
    """Buildear un JSON con la información de la data extraída
    """
    company = capitalizePalabras(company)
    flotas = sorted(df['Flota'].apply(capitalizePalabras).dropna().unique().tolist())
    oficinas = sorted(df['Oficina'].apply(capitalizePalabras).dropna().unique().tolist())
    codigos = sorted(df['Codigo'].dropna().unique().tolist())

    configDict = {
        "Cliente": company,
        "Informacion": {
            "Flotas": flotas,
            "Oficinas": oficinas,
            "Codigos": codigos
        },
    }

    return configDict

In [32]:
def loadJSON(path='metadata.json'):
    """Cargar el JSON con la metadata de las empresas
    devolviendo un diccionario de Python.
    """
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [34]:
# metadata = buildJSON(df, company="abastible_consolidado")
pathJSON = f'Clientes/{company}/metadata.json'

# with open(pathJSON, 'w', encoding='utf-8') as file:
#     json.dump(metadata, file, ensure_ascii=False, indent=2)

## Evolución - Gráfico de líneas
+ Total.
+ Por única flota.
+ Por *n* flotas.

In [12]:
def applyGroupEv(df, groupCols, freq='W'):
    """
    Agrupa el DataFrame df por las columnas en groupCols
    + la frecuencia semanal (freq) sobre 'Date'.
    Calcula:
    - raevSum = suma de raev
    - distanceSum = suma de distance
    - raev100 = (raevSum / distanceSum)*100
    
    Retorna columnas: [Date, *groupCols, raevSum, distanceSum, raev100]
    """
    grouped = (
        df.groupby([pd.Grouper(key='Date', freq=freq)] + groupCols)
            .agg(
                raevSum=('raev', 'sum'),
                distanceSum=('distance', 'sum')
        )
        .reset_index()
    )
    grouped = getRAEV100(grouped)
    grouped['Date'] = grouped['Date'] - timedelta(days=6)
    return grouped


def filterEv(df, startDate, endDate, flotas=None):
    """
    1) Sin flotas => columnas [Date, raev100].
    2) Lista de flotas con >1 => [Date, flota1, flota2, ...].
    3) Lista de flotas con 1 => [Date, raev100] (solo esa flota).
    Filtra también distance en (0,10000).
    """
    endDate = pd.to_datetime(endDate)
    endDate = endDate + pd.Timedelta(days=6)
    
    # Filtrado inicial
    df = df[(df['distance'] > 0) & (df['distance'] < 10000)]
    df = df[(df['Date'] >= startDate) & (df['Date'] <= endDate)]

    # Si hay flotas, filtrar
    if flotas:
        df = df[df['Flota'].isin(flotas)]

    # Caso A: Varias flotas => pivot
    if flotas and len(flotas) > 1:
        grouped = applyGroupEv(df, ['Flota'])
        grouped = getRAEV100Level(grouped, RIESGO['Bajo'], RIESGO['Medio'], RIESGO['Alto'])

        pivoted = grouped.pivot_table(
            index='Date', 
            columns='Flota', 
            values=['raev100', 'Riesgo'],
            aggfunc='first'
        ).reset_index()

        newCols = []
        for colTuple in pivoted.columns:
            if colTuple[0] == 'Date':
                newCols.append('Date')
            else:
                metric, flota = colTuple
                if metric == 'raev100':
                    newCols.append(f'{flota.capitalize()}-Raev100')
                else:
                    newCols.append(f'{flota.capitalize()}-Riesgo')
        pivoted.columns = newCols
        
        finalCols = ['Date']
        for flota in flotas:
            finalCols.append(f'{flota.capitalize()}-Raev100')
            finalCols.append(f'{flota.capitalize()}-Riesgo')

        return pivoted[finalCols].sort_values('Date')
    else:
        # Caso B: Cero flotas o 1 flota
        grouped = applyGroupEv(df, [])
        grouped = getRAEV100Level(grouped, RIESGO['Bajo'], RIESGO['Medio'], RIESGO['Alto'])
        dfRet = grouped[['Date', 'raev100', 'Riesgo']].sort_values('Date')
        
        if not flotas:
            dfRet.rename(columns={'raev100': 'Total-Raev100', 'Riesgo': 'Total-Riesgo'}, inplace=True)
        else:
            flotaName = flotas[0]
            dfRet.rename(columns={
                'raev100': f'{flotaName.capitalize()}-Raev100', 
                'Riesgo': f'{flotaName.capitalize()}-Riesgo'
                }, inplace=True)
        return dfRet

## Ranking - Gráfico de Barras
+ Total de flotas.
+ Total de oficinas.
+ Total de oficinas por *n* semanas.
+ Total de patentes.
+ Patentes de flota *a* y oficina *b*.
+ Patentes de flota *a* y oficina *b* en las últimas 4 semanas.
+ *Patentes más riesgosas (60 más riesgosas).

In [13]:
def applyGroup(df, groupCols, freq='W'):
    """
    Agrupa el DataFrame df por semana (según freq)
    y por las columnas en groupCols, calculando:
    - raevSum, distanceSum, dev
    - raev100 = (raevSum / distanceSum) * 100

    Devuelve un nuevo DataFrame con columnas:
    [Date, *groupCols, raevSum, distanceSum, dev, raev100].
    """
    grouped = (
        df.groupby([pd.Grouper(key='Date', freq=freq)] + groupCols)
        .agg(
            raevSum=('raev', 'sum'),
            distanceSum=('distance', 'sum'),
            dev=('dev', 'sum')
        ).reset_index()
    )
    grouped = getRAEV100(grouped)
    grouped['dev'] = grouped['dev'].round(0).astype(int)
    grouped['Date'] = grouped['Date'] - timedelta(days=6)
    return grouped


def pivotOficina(df):
    """
    Para el caso en que NO hay flotas (o flotas=None) y se desea
    la variación semanal por oficinas (ultimas != None).
    Hace un pivot con la columna 'raev100' a lo largo de las fechas,
    y agrega una columna 'dev' con la suma total de dev por oficina.
    """
    dfPivot = df.pivot(
        index='Oficina',
        columns='Date',
        values='raev100'
    ).reset_index()

    dfPivot.columns = (
        ['Oficina'] + [
            col.strftime('%d %b') if isinstance(col, pd.Timestamp) else col
            for col in dfPivot.columns[1:]
        ]
    )

    dfDev = (
        df.groupby('Oficina', as_index=False)['dev']
        .sum()
        .rename(columns={'dev': 'dev'})
    )

    return dfPivot.merge(dfDev, on='Oficina', how='left')

def joinCodigoOficina(df):
    """
    Si hay oficina y código, se unen en una nueva columna 'Oficina-Codigo'.
    """
    df['Oficina-Codigo'] = df['Oficina'] + '-' + df['Codigo']
    return df


def filterRanking(df, startDate=None, endDate=None, flotas=None, oficina=None, codigo=None, ultimas=None, top=None, dev=None):
    """
    Casos:
    1) Si hay endDate y ultimas => variación semanal (pivot):
        - Oficinas en filas, semanas en columnas, dev total.
    2) Si hay flotas:
        2.1) Varias flotas => [Flota, raev100, dev] (todas sumadas por semana).
        2.2) Una sola flota + oficina
            - Si también hay código => agrupa por semana => [Date, raev100, dev]
            - Sin código => agrupa por semana y 'Codigo' => [Codigo, raev100, dev]
        2.3) Una sola flota SIN oficina (nuevo subcaso)
            => agrupar por 'Oficina' => [Oficina, raev100, dev]
    3) Si no hay flotas pero no hay ultimas => ranking de oficinas (Oficina, raev100, dev).
    
    Si top está definido (int) => 
    Se ignora la lógica anterior y se devuelven
    los top 'N' códigos más riesgosos (por raev100) en ese período.
    """
    # --------------------------------------------------------------------------------
    # 1) Filtrado por fechas (startDate, endDate o ultimas)
    # --------------------------------------------------------------------------------
    if endDate:
        endDate = pd.to_datetime(endDate)
        df = df[df['Date'] <= endDate]

    if startDate:
        startDate = pd.to_datetime(startDate)
        df = df[df['Date'] >= startDate]
    else:
        # Si no hay startDate pero hay ultimas => 
        # se calcula el rango (endDate - ultimas semanas)
        if ultimas is not None:
            startDate = endDate - pd.DateOffset(weeks=ultimas) + pd.Timedelta(days=1)
            df = df[df['Date'] >= startDate]

    # --------------------------------------------------------------------------------
    # 2) Filtros adicionales (flotas, oficina, codigo)
    # --------------------------------------------------------------------------------
    if flotas:  # Si se pasa lista de flotas, filtramos
        df = df[df['Flota'].isin(flotas)]

    if oficina:  # Filtrar por oficina si aplica
        df = df[df['Oficina'] == oficina]

    if codigo:   # Filtrar por código si aplica
        df = df[df['Codigo'] == codigo]

    # --------------------------------------------------------------------------------
    # 3) Lógica de salida según parámetros recibidos
    # --------------------------------------------------------------------------------
    # Si top está definido => devolver los N códigos más riesgosos
    if top is not None and dev is not None:
        df = df[df['dev'] >= dev]
        df['Oficina'] = df['Oficina'].apply(capitalizePalabras)
        df = joinCodigoOficina(df)
        grouped = applyGroup(df, ['Codigo', 'Oficina-Codigo'])
        grouped = getRAEV100Level(grouped, RIESGO['Bajo'], RIESGO['Medio'], RIESGO['Alto'])
        grouped = grouped[['Oficina-Codigo', 'raev100', 'dev', 'Riesgo']].sort_values('raev100', ascending=False)
        return grouped.head(top)
    # Si HAY flotas
    if flotas:
        # Múltiples flotas
        if len(flotas) > 1:
            weekly = applyGroup(df, ['Flota'])
            final = (
                weekly.groupby('Flota', as_index=False)
                    .agg(
                        raevSum=('raevSum','sum'),
                        distanceSum=('distanceSum','sum'),
                        dev=('dev','sum')
                    )
            )
            final['raev100'] = ((final['raevSum'] / final['distanceSum']) * 100).round(1)
            final['Flota'] = final['Flota'].apply(capitalizePalabras)
            return final[['Flota', 'raev100', 'dev']].sort_values('raev100')
        
        # Una sola flota
        else:
            # Si oficina y código => agrupar por semana únicamente
            if oficina and codigo:
                grouped = applyGroup(df, [])
                grouped = getRAEV100Level(grouped, RIESGO['Bajo'], RIESGO['Medio'], RIESGO['Alto'])
                return grouped[['Date', 'raev100', 'dev', 'Riesgo']].sort_values('Date')
            
            # Si oficina y sin código => agrupar por semana, 'Codigo'
            elif oficina and not codigo:
                grouped = applyGroup(df, ['Codigo'])
                return grouped[['Codigo', 'raev100', 'dev']].sort_values('raev100')
            
            # Solo flota (sin oficina) => agrupar por 'Oficina'
            else:
                grouped = applyGroup(df, ['Oficina'])
                grouped['Oficina'] = grouped['Oficina'].apply(capitalizePalabras)
                grouped = getRAEV100Level(grouped, RIESGO['Bajo'], RIESGO['Medio'], RIESGO['Alto'])
                return grouped[['Oficina', 'raev100', 'dev', 'Riesgo']].sort_values('raev100', ascending=False)

    # Si NO hay flotas
    else:
        # Si ultimas está definido => variación semanal (pivot oficinas)
        if ultimas is not None:
            grouped = applyGroup(df, ['Oficina'])
            grouped['Oficina'] = grouped['Oficina'].apply(capitalizePalabras)
            grouped = pivotOficina(grouped)
            
            return grouped
        
        # Caso final => sumado total por semana y oficina, se muestran (Oficina, raev100, dev)
        grouped = applyGroup(df, ['Oficina'])
        grouped['Oficina'] = grouped['Oficina'].apply(capitalizePalabras)
        grouped = getRAEV100Level(grouped, RIESGO['Bajo'], RIESGO['Medio'], RIESGO['Alto'])
        return grouped[['Oficina','raev100','dev', 'Riesgo']].sort_values('raev100', ascending=False)

In [ ]:
tt = filterEv(df, '2024-01-29', '2025-01-20')
tt

In [81]:
# # 4
# ff = filterRanking(df, startDate='2025-01-20', endDate='2025-01-26', flotas=['TRANSFERENCIA', 'GRANEL', 'ENVASADO'])
# ff

In [161]:
# 5
ff = filterRanking(df, startDate='2025-01-20', endDate='2025-01-26')
ff

,Oficina,raev100,dev,Riesgo
18,Santiago,11.9,119,Muy Alto
16,Punta Arenas,11.7,143,Muy Alto
2,Chillan,8.7,138,Alto
12,Maipu,7.6,2532,Medio
0,Antofagasta,6.7,352,Medio
1,Arica,6.7,86,Medio
20,Temuco,6.4,632,Medio
17,San Fernando,6.4,571,Medio
8,Iquique,6.3,66,Medio
9,Lenga,6.2,1381,Medio


In [183]:
# 6
ff = filterRanking(df, endDate='2025-01-26', ultimas=2)
ff

,Oficina,13 ene.,20 ene.,dev
0,Antofagasta,6.5,6.7,683
1,Arica,6.7,6.7,141
2,Chillan,2.6,8.7,183
3,Chiloe,5.5,3.2,232
4,Concon,5.5,4.8,1397
5,Copiapo,5.3,4.5,219
6,Coyhaique,1.7,2.9,118
7,Curico,2.0,2.2,22
8,Iquique,3.1,6.3,92
9,Lenga,7.1,6.2,2987


In [98]:
df[(df['Date'] >= '2025-01-20') & (df['Date'] <= '2025-01-26') & (df['dev'] >= 50)]

,Oficina,Flota,Codigo,Date,bajo,medio,alto,distance,raev,dev
889,PEÑON,TRANSFERENCIA,EN6299,2025-01-25,4.9,6.2,8.2,892.899802,62.462836,52.975409
1006,PUNTA ARENAS,GRANEL,GR5055,2025-01-24,4.9,6.2,8.2,763.913652,140.357208,83.823961
1203,LENGA,TRANSFERENCIA,EN5859,2025-01-24,4.9,6.2,8.2,587.932904,81.523779,58.195198
1285,MAIPU,GRANEL,GR5017,2025-01-23,4.9,6.2,8.2,295.722123,85.197952,50.513628
1467,SAN FERNANDO,GRANEL,GR5021,2025-01-23,4.9,6.2,8.2,324.026836,95.951194,63.564119
1814,MAIPU,GRANEL,GR5023,2025-01-22,4.9,6.2,8.2,379.457934,91.197140,56.407454
1830,IQUIQUE,ENVASADO,EN3181,2025-01-22,4.9,6.2,8.2,322.873572,78.530436,54.762228
2030,TEMUCO,GRANEL,GR5049,2025-01-21,4.9,6.2,8.2,435.248498,90.184802,52.220901
2035,MAIPU,GRANEL,GR5054,2025-01-21,4.9,6.2,8.2,292.560150,109.985098,64.367151
2284,MAIPU,ENVASADO,EN6199,2025-01-20,4.9,6.2,8.2,273.131574,88.561025,69.011131


In [169]:
# 7
ff = filterRanking(df, startDate='2025-01-20', endDate='2025-01-26', top=60, dev=50)
ff

,Oficina-Codigo,raev100,dev,Riesgo
9,Maipu-GR5054,37.6,64,Muy Alto
3,Maipu-EN6199,32.4,69,Muy Alto
6,San Fernando-GR5021,29.6,64,Muy Alto
5,Maipu-GR5017,28.8,51,Muy Alto
0,Iquique-EN3181,24.3,55,Muy Alto
7,Maipu-GR5023,24.0,56,Muy Alto
1,Peñon-EN4587,23.7,85,Muy Alto
8,Temuco-GR5049,20.7,52,Muy Alto
10,Punta Arenas-GR5055,18.4,84,Muy Alto
2,Lenga-EN5859,13.9,58,Muy Alto


In [170]:
# 10
ff = filterEv(df, '2024-01-29', '2025-01-20', flotas=['ENVASADO'])
ff

,Date,Envasado-Raev100,Envasado-Riesgo
0,2024-01-29,9.2,Alto
1,2024-02-05,9.8,Muy Alto
2,2024-02-12,8.7,Alto
3,2024-02-19,10.2,Muy Alto
4,2024-02-26,9.4,Alto
5,2024-03-04,9.4,Alto
6,2024-03-11,9.7,Alto
7,2024-03-18,10.0,Muy Alto
8,2024-03-25,10.1,Muy Alto
9,2024-04-01,9.1,Alto


In [175]:
# 11
ff = filterRanking(df, startDate='2025-01-20', endDate='2025-01-26', flotas=['ENVASADO'])
ff

,Oficina,raev100,dev,Riesgo
15,Puerto Montt,11.5,106,Muy Alto
9,Lenga,9.9,424,Muy Alto
8,Iquique,9.7,60,Alto
2,Chillan,8.7,138,Alto
14,Peñon,7.9,400,Medio
0,Antofagasta,7.6,197,Medio
12,Maipu,7.6,730,Medio
19,Valdivia,6.7,95,Medio
17,Talca,6.7,113,Medio
18,Temuco,6.4,218,Medio


In [178]:
# 13
ff = filterRanking(df, startDate=None, endDate='2025-01-26', flotas=['ENVASADO'], oficina='ANTOFAGASTA', codigo='EN3807', ultimas=4)
ff

,Date,raev100,dev,Riesgo
0,2024-12-30,12.4,92,Muy Alto
1,2025-01-06,11.0,81,Muy Alto
2,2025-01-13,14.5,107,Muy Alto
3,2025-01-20,17.0,99,Muy Alto


# PDF

In [14]:
def buildFolder(company, week, base_path='./Clientes/'):
    """Crear carpeta para guardar los archivos
    de cada Cliente y semana.
    """
    path = os.path.join(base_path, company, week)
    subfolders = ['table', 'png']
    
    for subfolder in subfolders:
        os.makedirs(os.path.join(path, subfolder), exist_ok=True)
    
    return path

def exportToPNG(pdfPath, excludePages, outputFolder, company):
    """Exportar todas las páginas de un PDF a PNG
    excepto las que están en excludePages.
    """
    doc = pymupdf.open(pdfPath)
    
    for numPage in range(len(doc)):
        if numPage not in excludePages:
            page = doc.load_page(numPage)
            render = page.get_pixmap()
            render.save(os.path.join(outputFolder, f"{company}_Page_{numPage + 1}.png"))
    
    doc.close()

In [15]:
week = '2025-01-20'

In [16]:
path = buildFolder(company, week)

In [17]:
pdfName = 'Abastible'
pdfPath = f'Clientes/{company}/{week}/{pdfName}.pdf'
outputFolder = os.path.join(path, 'png')
excludePages = [0, 7, 8, 21, 35]

In [18]:
def buildCSVfromJSON(data: pd.DataFrame, jsonPath: str, outputDir: str = f"./Clientes/{company}/{week}/table/"):
    """Leer el config JSON, y para cada cliente llamar a 
    la función de filter adecuada según los parámetros.
    """
    os.makedirs(outputDir, exist_ok=True)

    with open(jsonPath, "r", encoding="utf-8") as f:
        config = json.load(f)

    for cliente in config["Clientes"]:
        nombreCliente = cliente["Nombre"]

        for grafico in cliente["Gráficos"]:
            numPage = grafico.get("Pagina", 1)
            tipo = grafico["Tipo"]
            
            for filtrosDict in grafico["Filtros"]:
                if "ultimas" in filtrosDict:
                    filtrosDict["ultimas"] = int(filtrosDict["ultimas"])
                if "top" in filtrosDict:
                    filtrosDict["top"] = int(filtrosDict["top"])

                if tipo == "Evolucion":
                    dfResult = filterEv(
                        data,
                        startDate=filtrosDict.get("startDate"),
                        endDate=filtrosDict.get("endDate"),
                        flotas=filtrosDict.get("Flotas")
                    )
                elif tipo == "Ranking":
                    dfResult = filterRanking(
                        data,
                        startDate=filtrosDict.get("startDate"),
                        endDate=filtrosDict.get("endDate"),
                        flotas=filtrosDict.get("Flotas"),
                        oficina=filtrosDict.get("Oficina"),
                        codigo=filtrosDict.get("Codigo"),
                        ultimas=filtrosDict.get("ultimas"),
                        top=filtrosDict.get("top"),
                        dev=filtrosDict.get("dev")
                    )
                else:
                    print(f"[Advertencia] Tipo de gráfico no reconocido: {tipo}")
                    continue

                csvFile = f"{nombreCliente}_Page_{numPage}.csv"
                outputPath = os.path.join(outputDir, csvFile)
                
                dfResult.to_csv(outputPath, index=False, encoding="utf-8")
                
                print(f"Generado CSV: {outputPath}")

In [17]:
exportToPNG(pdfPath, excludePages, outputFolder, company)

In [199]:
buildCSVfromJSON(df, "config.json")

Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_2.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_3.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_4.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_5.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_6.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_7.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_10.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_11.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_12.csv
Generado CSV: ./Clientes/abastible_consolidado/2025-01-20/table/abastible_consolidado_Page_13.csv
Generado CSV: ./Clientes/a

# OpenAI

In [19]:
import openai
import time
from dotenv import load_dotenv

In [20]:
load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
ASSISTANT_ID = os.getenv("OPENAI_ASSISTANT_ID")

In [21]:
client = openai.OpenAI()

In [26]:
# assistant = client.beta.assistants.create(
#     model="gpt-4o",
#     name="Analista RAEV100",
#     description="Genera observaciones precisas y concisas sobre reportes semanales de RAEV/100.",
#     instructions="""1. Rol principal y priorización de datos numéricos (CSV sobre imágenes):
#     "Este GPT es un analista de reportes de riesgo de accidentes basado en los índices de RAEV/100, priorizando la interpretación de datos numéricos en archivos CSV por sobre los gráficos asociados en imágenes para no fallar en la identificación de números, también en caso de existir un csv con estadística descriptiva, utiliza esto para generar observaciones más potentes en un párrafo 2 o 3 líneas como máximo. Se otorgará mayor peso al análisis del CSV para observaciones relevantes."

#     2. Contexto de reportes semanales y comparación con semanas anteriores:
#     "Para que las observaciones que se busque generar sean precisas y concisas sobre los nuevos reportes semanales que se le irán entregando, tendrá que comparar la semana con respecto a las anteriores y su comportamiento.
#     Cuando se hable de "códigos" se habla de patentes."

#     3. Definición e importancia del índice RAEV/100:
#     "El índice RAEV/100 es un índice que acumula el riesgo que va generando un vehículo y/o un conductor en un periodo de tiempo determinado. Puede presentarse en términos absolutos o relativos a 100 km. La agregación del RAEV de los vehículos de una flota, representa el riesgo generado por la flota en el periodo de tiempo. El RAEV/100 considera la relación entre el límite de velocidad en un tramo con la velocidad que lleva el vehículo, por lo cual recorrer muchos kilómetros en exceso de velocidad (Km Ev) por muy cerca del límite de velocidad, no es tan riesgoso como recorrer kilómetros en exceso de velocidad a mucha mayor velocidad de lo que el límite permite. También considera el tiempo en que anduvo en ese exceso, si es por mucho tiempo es mucho más riesgoso y si es por muy poco tiempo no es tan riesgoso. El índice RAEV y los algoritmos que lo calculan no es algo de conocimiento público en internet pero se basa en lo siguiente:"

#     4. Cálculo de eventos de exceso de velocidad (EV) y factores que inciden en el riesgo:
#     "Que es posible conocer si un vehículo se encuentra a exceso de velocidad si conocemos:

#     El tipo de vehículo (Ej. Camión, Camioneta, Bus).
#     El tipo de carga que transporta (Ej. Peligrosa, líquida, Pasajeros).
#     El lugar específico de una carretera en que se encuentra (reportado por el GPS).
#     La velocidad instantánea que lleva el vehículo (reportado por el GPS).
#     El sentido de desplazamiento que lleva (deducido por los reportes del GPS).
#     La regulación específica que la ley define en ese tramo y sentido, a ese tipo de vehículo y ese tipo de carga (definición entregada por VIALIDAD y ley del tránsito, en el caso de Chile).
#     Que para cada evento de exceso de velocidad podemos calcular el riesgo de accidente que se genera, el que en términos generales dependerá de:

#     Las características específicas del evento. Ejemplo: distancia recorrida, curva de velocidad durante el evento, duración del evento, entre otras varias.
#     Las circunstancias específicas en que ocurre el evento (tipo de carga, nivel de carga, nivel de congestión de la carretera, hora del día, nivel de cansancio del conductor, entre otros)."

#     5. Definición de 'Km Ev':
#     "Cuando se mencione 'Km Ev', se interpretará como kilómetros recorridos en exceso de velocidad."

#     6. Formato y extensión de las observaciones (2 a 3 líneas, directas y concisas):
#     "Las observaciones serán directas y concisas, limitadas a un párrafo de 2 o 3 líneas como máximo, enfocándose exclusivamente en los puntos clave de riesgo y variaciones detectadas. No se redundará en descripciones innecesarias del contexto o de las tablas analizadas, sino que se apuntará directamente a las conclusiones más relevantes, destacando incrementos, disminuciones o riesgos significativos."

#     7. Reglas obligatorias y recordatorio general:
#     Mantener la instrucción de comparar la nueva semana con semanas anteriores.
#     No omitir la concentración en datos numéricos (CSV) ni el uso de la estadística descriptiva si está disponible.
#     Responder únicamente con la observación directa, precisa y concisa (2 o 3 líneas máximo).
#     Limitarse a responder solamente con la observación como párrafo de 2 o 3 líneas como máximo.""",
#     tools=[{"type": "code_interpreter"}]
# )

# print(assistant.id)

In [78]:
updateInstruction = """1. Rol principal y priorización de datos numéricos (CSV sobre imágenes):
"Este GPT es un analista de reportes de riesgo de accidentes que entrega observaciones precisas y concisas en un solo párrafo de 2 o 3 líneas como máximo basado en los índices de RAEV/100, priorizando la interpretación de datos numéricos en tablas en formato CSV o texto plano por sobre los gráficos asociados en imágenes PNG para no fallar en la identificación de números, también en caso de existir un csv con estadística descriptiva, utilizar esto para generar observaciones más potentes. Se otorgará mayor peso al análisis de la tabla CSV para observaciones relevantes."

2. Contexto de reportes semanales y comparación con semanas anteriores:
"-Para que las observaciones que se busque generar sean precisas y concisas sobre los nuevos reportes semanales que se le irán entregando, tendrá que comparar la semana con respecto a las anteriores y su comportamiento, esto en caso de tener información sobre las semanas anteriores en la tabla CSV. 
-Cuando se hable de "códigos" se habla de patentes."

3. Definición e importancia del índice RAEV/100:
"El índice RAEV/100 es un índice que acumula el riesgo que va generando un vehículo y/o un conductor en un periodo de tiempo determinado. Puede presentarse en términos absolutos o relativos a 100 km, pero siempre por cada 100 km. La agregación del RAEV de los vehículos de una flota, representa el riesgo generado por la flota en el periodo de tiempo. El RAEV/100 considera la relación entre el límite de velocidad en un tramo con la velocidad que lleva el vehículo, por lo cual recorrer muchos kilómetros en exceso de velocidad (Km Ev) por muy cerca del límite de velocidad, no es tan riesgoso como recorrer kilómetros en exceso de velocidad a mucha mayor velocidad de lo que el límite permite. También considera el tiempo en que anduvo en ese exceso, si es por mucho tiempo es mucho más riesgoso y si es por muy poco tiempo no es tan riesgoso. El índice RAEV y los algoritmos que lo calculan no es algo de conocimiento público en internet pero se basa en lo siguiente:"

4. Cálculo de eventos de exceso de velocidad (EV) y factores que inciden en el riesgo:
"Que es posible conocer si un vehículo se encuentra a exceso de velocidad si conocemos:

- El tipo de vehículo (Ej. Camión, Camioneta, Bus).
- El tipo de carga que transporta (Ej. Peligrosa, líquida, Pasajeros).
- El lugar específico de una carretera en que se encuentra (reportado por el GPS).
- La velocidad instantánea que lleva el vehículo (reportado por el GPS).
- El sentido de desplazamiento que lleva (deducido por los reportes del GPS).
- La regulación específica que la ley define en ese tramo y sentido, a ese tipo de vehículo y ese tipo de carga (definición entregada por VIALIDAD y ley del tránsito, en el caso de Chile).
Que para cada evento de exceso de velocidad podemos calcular el riesgo de accidente que se genera, el que en términos generales dependerá de:

- Las características específicas del evento. Ejemplo: distancia recorrida, curva de velocidad durante el evento, duración del evento, entre otras varias.
- Las circunstancias específicas en que ocurre el evento (tipo de carga, nivel de carga, nivel de congestión de la carretera, hora del día, nivel de cansancio del conductor, entre otros)."

5. Definición de 'Km Ev':
- Los 'Km Ev', se interpretará como kilómetros recorridos en exceso de velocidad.
- En las tablas CSV o texto plano esta información viene en las columnas 'dev'. Entonces 'dev' es lo mismo que 'Km Ev'.

6. Formato y extensión de las observaciones (2 a 3 líneas, directas y concisas):
- Las observaciones serán directas y concisas, limitadas a un párrafo de 2 o 3 líneas como máximo, enfocándose exclusivamente en los puntos clave de riesgo y variaciones detectadas.
- No se redundará en descripciones innecesarias del contexto o de las tablas analizadas, sino que se apuntará directamente a las conclusiones más relevantes, destacando incrementos, disminuciones o riesgos significativos.
- Cuando se den las observaciones, se empezará directo a la conclusión, sin dar rodeos, como por ejemplo empezar por: "En la semana X, se observa que..." sino que se irá directo a la observación relevante.
- No mencionar la semana en la que se está observando, ya que se asume que se está observando esa semana en particular. Simplemente ir directo a la observación relevante.
- No mencionar la empresa o cliente en la observación, ya que se asume que se está observando la empresa o cliente en cuestión. Simplemente ir directo a la observación relevante.
- Empezar las observaciones directamente con la conclusión, sin dar rodeos, como por ejemplo empezar por: "En la semana X, se observa que..." sino que se irá directo a la observación relevante, ejemplo: "Se observa que..." o "Se destaca que..." o "Es relevante mencionar que..." o "Es importante notar que...", "La flota X presenta un riesgo alto...", entre otros.
- Las observaciones serán coherentes para todo el contexto dado, destacando la relación entre el RAEV/100 y los Km Ev en caso de tener esta información en las tablas CSV.
- Si se presenta una columna "Oficina-Codigo" en las tablas CSV, se debe referir a esta columna como "El código X de la oficina Y" o "El código X de la oficina Z" y no como "El código X de la oficina-código Y" o "Oficina-Codigo es el valor con mayor RAEV/100".
- Haz observaciones que destaquen TODA la información relevante, no concentrarse solo en "los más altos" o "los más bajos", sino en toda la información relevante que se pueda extraer de las tablas CSV.
- En caso de contar con la columna de Riesgo (Bajo, Medio, Alto, Muy Alto) en las tablas CSV o texto plano, utiliza esta información para generar observaciones más potentes y con coherencia con respecto al RAEV/100 en un párrafo de 2 o 3 líneas como máximo.
- Si se cuenta con la columna de Riesgo (Bajo, Medio, Alto, Muy Alto) en las tablas CSV o texto plano, refierete a los niveles de riesgo de forma natural en tus observaciones, es decir, mencionar el nivel sin comillas ni mayúsculas, sino de forma natural, como por ejemplo: "En la semana X, se observa que el riesgo es bajo en comparación a la semana anterior" y no decir "En la semana X, se observa que el riesgo es 'Bajo' en comparación a la semana anterior" o "En la semana X, se observa que el riesgo es Bajo en comparación a la semana anterior". Aplica esto SÓLO para los niveles de riesgo.
- En caso de existir información sobre Km Ev, 'dev' en las tablas CSV, destaca casos en los que pueda existir un alto valor en Km Ev con un bajo RAEV/100 o viceversa en un párrafo de 2 o 3 líneas como máximo.
- No entregar como observación los datos numéricos en sí, si no que destacar la tendencia, interpretación y relación entre los datos.
- La columna de Riesgo (Bajo, Medio, Alto, Muy Alto) en las tablas CSV o texto plano, está directamente relacionada con el RAEV/100, por lo que si se cuenta con esta información, utilizarla para generar observaciones más potentes y con coherencia con respecto al RAEV/100 en un párrafo de 2 o 3 líneas como máximo.
- Una semana, flota, oficina, código o código-patente, puede tener un nivel de riesgo (Bajo, Medio, Alto, Muy Alto) igual a otra semana, flota, oficina, código o código-patente, pero tener un RAEV/100 diferente. Significa que están en el mismo umbral de riesgo pero con diferente RAEV/100.
- En caso de existir varias semanas en la tabla CSV o texto plano, recuerda que la semana objetivo siempre será la actual revelada en el prompt, por lo que se asume que se está observando la semana actual en particular.
- Comparar la semana actual con las anteriores en la tabla CSV o texto plano, en caso de tener información sobre las semanas anteriores.

7. Reglas obligatorias y recordatorio general:
- Los cálculos de RAEV/100 y todos los datos numéricos entregados en las tablas CSV o texto plano, son los datos oficiales y correctos, por lo que no se deben cuestionar estos datos, sino que interpretarlos y analizarlos de forma correcta.
- Mantener la instrucción de comparar la semana solicitada a observar con las semanas anteriores, en caso de tener información sobre las anteriores en la tabla CSV.
- No omitir la concentración en datos numéricos (CSV) ni el uso de la estadística descriptiva si está disponible.
- Responder únicamente con la observación directa, precisa y concisa (2 o 3 líneas máximo).
- Limitarse a responder solamente con la observación como párrafo de 2 o 3 líneas como máximo.
- NO mencionar la semana en la que se está observando, ya que se asume que se está observando esa semana en particular. Simplemente ir directo a la observación relevante.
- En caso de necesitar decir la semana, se puede referir a ella como "esta semana" o "la semana actual" pero no como "En la semana iniciada el YYYY-MM-DD".
- NUNCA referirse a Km Ev como 'dev' si no que siempre referirse como Km Ev o kilómetros en exceso de velocidad o kilómetros recorridos en exceso de velocidad.
- Si algún dato es NaN o nulo, simplemente omitirlo y seguir con la observación directa y concisa en un párrafo de 2 o 3 líneas como máximo. Si es necesario o relevante referirse a ellos, decir como por ejemplo "No se obtuvieron registros..." o "No se cuenta con información..." pero no referirse a ellos como "NaN" o "Nulo" o "No hay datos".
- NO referirse a la observación como "El reporte semanal" o "Este reporte semanal" o "El informe semanal" o "Este informe semanal" sino que simplemente ir directo a la observación relevante.
- En caso de tener errores, la última parte de la respuesta debe tener si o si y sólo la observación como párrafo de 2 o 3 líneas como máximo.
- En caso de faltar información, como por ejemplo: registro de semanas anteriores, estadísticas descriptivas en otro CSV, entre otros, NO indiques que no tienes esa información, simplemente omite esa parte y sigue con la observación directa y concisa en un párrafo de 2 o 3 líneas como máximo.
- Hay veces que se adjuntará el CSV y otras veces se pasará el CSV en el prompt como texto plano. En ambos casos, se espera que la observación sea coherente con la información entregada en el CSV o en el prompt."""

In [23]:
tempUpdate = 0.7

In [79]:
assistant = client.beta.assistants.update(
    assistant_id=ASSISTANT_ID,
    instructions=updateInstruction,
    temperature=tempUpdate,
    model="gpt-4o-mini"
)

In [ ]:
# myAssistant = client.beta.assistants.list()

# print(myAssistant.data)

[Assistant(id='asst_5EG4HVwef686wwp6X5HHs689', created_at=1737648152, description='Genera observaciones precisas y concisas sobre reportes semanales de RAEV/100.', instructions='1. Rol principal y priorización de datos numéricos (CSV sobre imágenes):\n"Este GPT es un analista de reportes de riesgo de accidentes que entrega observaciones precisas y concisas en un solo párrafo de 2 o 3 líneas como máximo basado en los índices de RAEV/100, priorizando la interpretación de datos numéricos en tablas en formato CSV por sobre los gráficos asociados en imágenes PNG para no fallar en la identificación de números, también en caso de existir un csv con estadística descriptiva, utilizar esto para generar observaciones más potentes. Se otorgará mayor peso al análisis de la tabla CSV para observaciones relevantes."\n\n2. Contexto de reportes semanales y comparación con semanas anteriores:\n"-Para que las observaciones que se busque generar sean precisas y concisas sobre los nuevos reportes semanales

In [25]:
def csvToText(csvPath):
    """Convertir un archivo CSV a texto plano
    """
    df = pd.read_csv(csvPath)
    return df.to_string(index=False)

def pngBase64(pngPath):
    with open(pngPath, "rb") as img:
        return b64encode(img.read()).decode("utf-8")

def GetFileId(filePath, purp = "assistants"):
    """Subir un archivo y obtener su file_id."""
    with open(filePath, "rb") as file:
        response = client.files.create(
            file=file,
            purpose=purp
        )
    return response.id

In [75]:
def countTokens(text, modelName="gpt-4o-mini"):
    """Contar los tokens de un texto según el modelo."""
    encoding = tiktoken.encoding_for_model(modelName)
    return len(encoding.encode(text))

def generateObservation(csvPath, prompt):
    """Generar una observación a partir de un archivo CSV y una imagen PNG
    """
    try:
        thread = client.beta.threads.create()
        
        message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=prompt
        )
        run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=ASSISTANT_ID
        )

        print("Run completed with status: " + run.status)
        
        messages = None
        if run.status == "completed":
            promptTokens = countTokens(prompt)
            
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            
            outputTokens = 0

            print("messages: ")
            for message in messages:
                #assert message.content[0].type == "text"
                try:
                    print({"role": message.role, "message": message.content[0].text.value.replace("Santiago", "Disponibles").replace("SANTIAGO", "DISPONIBLES")})
                    outputTokens += countTokens(message.content[0].text.value)
                except:
                    break
            print(f'Tokens input: {promptTokens} - Tokens output: {outputTokens}')
        else:
            print(f'Error en la generación de observación: {run.last_error}')
        return run, messages, message
    except Exception as e:
        print(f'Error en la generación de observación: {e}')
    
    return None, None, None

In [27]:
def saveObservation(history, jsonFile):
    decision = ""
    while decision not in ["y", "n"]:
        decision = input("Guardar observación? (y/n): ").strip().lower()
        if decision == "y":
            if os.path.exists(jsonFile):
                with open(jsonFile, "r", encoding="utf-8") as f:
                    oldHistory = json.load(f)
            else:
                oldHistory = []

            for interaction in history:
                if interaction not in oldHistory:
                    oldHistory.append(interaction)

            with open(jsonFile, "w", encoding="utf-8") as f:
                json.dump(oldHistory, f, ensure_ascii=False, indent=4)

In [28]:
jsonFile = "history.json"

In [76]:
grafico = f"{company}_Page_28"
csvPath = f"Clientes/{company}/{week}/table/{grafico}.csv"
pngPath = f"Clientes/{company}/{week}/png/{grafico}.png"

csv = csvToText(csvPath)
jsonDict = loadJSON(pathJSON)
# png = pngBase64(pngPath)

# findWeek = datetime.strptime(week, '%Y-%m-%d')
# finalDay = findWeek + timedelta(days=6)

# weekPrompt = finalDay.strftime('%Y-%m-%d')

# weekPrompt = '2024-04-28'

prompt = f"""Genera una observación sobre este reporte semanal de riesgo RAEV/100 de la empresa {company} correspondiente a la semana iniciada el {week} utilizando la tabla CSV en texto plano. En caso de existir información sobre semanas anteriores, compara la semana iniciada el {week} con las anteriores, si no existe información sobre las anteriores, omite este paso.
{csv}

Contexto de la empresa:
{jsonDict}
"""

obs = generateObservation(csvPath, prompt)
run, messages, message = obs

Run completed with status: completed
messages: 
{'role': 'assistant', 'message': 'Se observa que el RAEV/100 se mantiene en un nivel bajo de 5.2, similar al de la semana anterior, pero con un incremento en los kilómetros recorridos en exceso de velocidad, alcanzando 47 km Ev. Esto sugiere una tendencia que podría aumentar el riesgo si se continúa este comportamiento.'}
{'role': 'user', 'message': "Genera una observación sobre este reporte semanal de riesgo RAEV/100 de la empresa abastible_consolidado correspondiente a la semana iniciada el 2025-01-20 utilizando la tabla CSV en texto plano. En caso de existir información sobre semanas anteriores, compara la semana iniciada el 2025-01-20 con las anteriores, si no existe información sobre las anteriores, omite este paso.\n      Date  raev100  dev Riesgo\n2024-12-30      5.1   35   Bajo\n2025-01-06      5.3   26   Bajo\n2025-01-13      4.8   52   Bajo\n2025-01-20      5.2   47   Bajo\n\nContexto de la empresa:\n{'Cliente': 'Abastible_conso

In [44]:
csvTest = pd.read_csv(csvPath)
csvTest

,Date,raev100,dev,Riesgo
0,2024-12-30,5.1,35,Bajo
1,2025-01-06,5.3,26,Bajo
2,2025-01-13,4.8,52,Bajo
3,2025-01-20,5.2,47,Bajo


In [193]:
assistantResponse = None
for msg in reversed(messages.data):
    if msg.role == "assistant":
        assistantResponse = msg.content[0].text.value.replace("Santiago", "Disponibles").replace("SANTIAGO", "DISPONIBLES")
        break

print('Observación: ' + assistantResponse)

Observación: Disponibles presenta una significativa disminución en el índice RAEV/100 a pesar de registrar un alto valor de Km Ev, lo que sugiere que el riesgo se está gestionando eficientemente. En contraste, Chillan experimenta un notable aumento en el RAEV/100, indicando un incremento en el riesgo asociado. Además, Maipú mantiene un alto índice RAEV/100 y registra el mayor valor de Km Ev, señalando un área crítica en términos de riesgo.


In [191]:
if "history" not in locals():
    history = []

if "currentInteraction" not in locals():
    currentInteraction = {"user": [], "assistant": []}

newInteractions = []

for message in reversed(messages.data):
    if message.role == "user":
        # Si hay datos en "assistant", guardar la interacción actual y empezar una nueva
        if currentInteraction["assistant"]:
            newInteractions.append(currentInteraction)
            currentInteraction = {"user": [], "assistant": []}
        # Agregar mensaje del usuario
        currentInteraction["user"].append(message.content[0].text.value)
    elif message.role == "assistant":
        # Agregar mensaje del asistente
        currentInteraction["assistant"].append(message.content[0].text.value.replace("Santiago", "Disponibles").replace("SANTIAGO", "DISPONIBLES"))

# Guardar la última interacción procesada si tiene datos
if currentInteraction["user"] or currentInteraction["assistant"]:
    newInteractions.append(currentInteraction)

# Evitar duplicados comparando con el historial existente
for interaction in newInteractions:
    if interaction not in history:
        history.append(interaction)

saveObservation(history, jsonFile)

In [56]:
# print(json.dumps(history, indent=4, ensure_ascii=False))

In [43]:
rr = filterRanking(df, startDate=None, endDate='2025-01-26', flotas=['ENVASADO'], oficina='ANTOFAGASTA', codigo='EN3807', ultimas=4)
rr

,Date,raev100,dev
0,2024-12-30,12.4,92
1,2025-01-06,11.0,81
2,2025-01-13,14.5,107
3,2025-01-20,17.0,99


In [220]:
# 14
# kkk = filterRanking(df, None, '2024-04-22', ['ENVASADO'], oficina='ANTOFAGASTA', codigo='EN3800', ultimas=4)
# kkk

,Date,raev100,dev
0,2024-04-01,6.3,39
1,2024-04-08,4.0,14
2,2024-04-15,4.4,42
3,2024-04-22,5.4,26


In [121]:
# Sacar promedio de Envasado sin contar la semana 2024-04-21 y 2024-04-28
csvTest = csvTest[(csvTest['Date'] != '2024-04-21') & (csvTest['Date'] != '2024-04-28')]
csvTest['TRANSFERENCIA'].mean()

8.110149805336915

# Observaciones en PDF

In [196]:
def insertObservationsPDF(
    pdfPath: str,
    outputPDF: str,
    csvFolder: str,
    excludePages: list,
    company: str,
    week: str,
    marginTop: float = 15,
    marginLeft: float = 60,
    textBoxHeight: float = 140
):
    doc = pymupdf.open(pdfPath)
    totalPages = len(doc)
    
    for p in range(totalPages):
        if p in excludePages:
            print(f'Omitiendo página {p + 1}')
            continue
        page = doc.load_page(p)

        pageNumber = p + 1
        csvName = f"{company}_Page_{pageNumber}.csv"
        csvPath = os.path.join(csvFolder, csvName)

        if not os.path.exists(csvPath):
            print(f"No se encontró el archivo CSV: {csvName}. Saltando la inserción de la página {pageNumber + 1}")
            continue
        
        csvText = csvToText(csvPath)

        prompt = f"""Genera una observación sobre este reporte semanal de riesgo RAEV/100 de la empresa {company.capitalize()} correspondiente a la semana iniciada el {week} utilizando la tabla CSV en texto plano. En caso de existir información sobre semanas anteriores, compara la semana iniciada el {week} con las anteriores, si no existe información sobre las anteriores, omite este paso.
        {csvText}

        Contexto de la empresa:
        {jsonDict}
        """
        run, messages, message = generateObservation(csvPath, prompt)

        assistantResponse = None
        for msg in reversed(messages.data):
            if msg.role == "assistant":
                assistantResponse = msg.content[0].text.value.replace("Santiago", "Disponibles").replace("SANTIAGO", "DISPONIBLES")
                break
        if not assistantResponse:
            print(f"No se encontró respuesta del asistente para la página {pageNumber + 1}. Saltando la inserción.")
            continue

        width = page.rect.width
        height = page.rect.height

        pos = pymupdf.Rect(
            marginLeft,                            # x0
            height - marginTop - textBoxHeight,    # y0
            width - marginLeft,                    # x1
            height - marginTop,                    # y1
        )

        print(f"Escribiendo observación en página {pageNumber + 1}")

        page.insert_htmlbox(
            pos,
            '<b>Observación:</b> ' + assistantResponse,
            css= "* {font-family: helvetica; font-size: 24px; color: #000000;}",
        )
    doc.save(outputPDF)
    doc.close()
    print(f"PDF guardado en: {outputPDF}")

In [80]:
def generateObsAsync(args):
    """Generar observaciones de forma asíncrona
    """
    numPage = args['page']
    csvPath = args['csvPath']
    prompt = args['prompt']
    
    run, messages, message = generateObservation(csvPath, prompt)
    if run is None:
        return (numPage, "Error: no se realizó la respuesta en el asistente")
    assistantResponse = None
    if run.status == "completed":
        for msg in reversed(messages.data):
            if msg.role == "assistant":
                assistantResponse = msg.content[0].text.value.replace("Santiago", "Disponibles").replace("SANTIAGO", "DISPONIBLES")
                break
    else:
        assistantResponse = "Error: no se completó la respuesta"

    return (numPage, assistantResponse)

def insertObsPDF(
    pdfPath: str,
    outputPDF: str,
    csvFolder: str,
    excludePages: list,
    company: str,
    week: str,
    marginTop: float = 15,
    marginLeft: float = 60,
    textBoxHeight: float = 140
):
    doc = pymupdf.open(pdfPath)
    totalPages = len(doc)

    # Crear lista de prompts
    tasks = []
    for p in range(totalPages):
        if p in excludePages:
            continue
        numPage = p + 1
        csvName = f"{company}_Page_{numPage}.csv"
        csvPath = os.path.join(csvFolder, csvName)
        if not os.path.exists(csvPath):
            continue

        csvText = csvToText(csvPath)

        prompt = f"""Genera una observación sobre este reporte semanal de riesgo RAEV/100 de la empresa {company.capitalize()} correspondiente a la semana iniciada el {week} utilizando la tabla CSV en texto plano. En caso de existir información sobre semanas anteriores, compara la semana iniciada el {week} con las anteriores, si no existe información sobre las anteriores, omite este paso.
        {csvText}

        Contexto de la empresa:
        {jsonDict}
        """

        tasks.append({"page": p, "csvPath": csvPath, "prompt": prompt})

    # Lanzar en paralelo
    responses = {}  # dict: pageIndex -> assistantResponse
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        futureToPage = {
            executor.submit(generateObsAsync, t): t["page"] for t in tasks
        }
        for future in concurrent.futures.as_completed(futureToPage):
            pageIndex = futureToPage[future]
            try:
                page_num, assistantResponse = future.result()
                responses[page_num] = assistantResponse
            except Exception as exc:
                responses[pageIndex] = f"Error: {exc}"

    # Insertar en PDF en orden
    for p in range(totalPages):
        if p in excludePages:
            print(f'Omitiendo página {p + 1}')
            continue
        if p not in responses:
            print(f"No se encontró respuesta del asistente para la página {p + 1}. Saltando la inserción.")
            continue
        assistantResponse = responses[p]
        if not assistantResponse:
            print(f"No se encontró respuesta del asistente para la página {p + 1}. Saltando la inserción.")
            continue

        page = doc.load_page(p)
        width = page.rect.width
        height = page.rect.height

        pos = pymupdf.Rect(
            marginLeft,
            height - marginTop - textBoxHeight,
            width - marginLeft,
            height - marginTop,
        )
        
        fontName = "Montserrat-Regular"
        fontFile = f"fonts/{fontName}.ttf"
        page.insert_font(fontname=fontName, fontfile=fontFile)
        
        print(f"Escribiendo observación en página {p + 1}")
        try:
            page.insert_htmlbox(
                pos,
                '<b>Observación:</b> ' + assistantResponse,
                css=f"* {{font-family: {fontName}; font-size: 24px; color: #000000;}}",
            )
        except Exception as e:
            print(f"Error al insertar observación en página {p + 1}: {e}")
        print(f"Observación insertada en página {p + 1}")

    doc.save(outputPDF)
    doc.close()
    print(f"PDF guardado en: {outputPDF}")

In [81]:
pdfName = 'Abastible'
pdfPath = f'Clientes/{company}/{week}/{pdfName}.pdf'

# insertObservationsPDF(
#     pdfPath,
#     f"Clientes/{company}/{week}/{pdfName}_ready.pdf",
#     f"Clientes/{company}/{week}/table",
#     excludePages,
#     company,
#     week
# )

model = "gpt-4o-mini"

insertObsPDF(
    pdfPath,
    f"Clientes/{company}/{week}/{pdfName}_{model}_4.pdf",
    f"Clientes/{company}/{week}/table",
    excludePages,
    company,
    week
)

Run completed with status: completed
Run completed with status: completed
messages: 
{'role': 'assistant', 'message': 'Se destaca que la oficina de Lenga presenta el mayor riesgo, clasificado como muy alto, con un RAEV/100 de 9.9 y 424 kilómetros recorridos en exceso de velocidad. En contraste, la oficina de San Fernando tiene el riesgo más bajo, pero mantiene un nivel medio de 5.8 en RAEV/100, sugiriendo una variabilidad considerable en el comportamiento de riesgo entre las distintas oficinas.'}
{'role': 'user', 'message': "Genera una observación sobre este reporte semanal de riesgo RAEV/100 de la empresa Abastible_consolidado correspondiente a la semana iniciada el 2025-01-20 utilizando la tabla CSV en texto plano. En caso de existir información sobre semanas anteriores, compara la semana iniciada el 2025-01-20 con las anteriores, si no existe información sobre las anteriores, omite este paso.\n             Oficina  raev100  dev   Riesgo\nPuerto Montt     11.5  106 Muy Alto\n       L

In [201]:
gpt = f'{company}_Page_46.csv'
gptTable = pd.read_csv(f'./Clientes/{company}/{week}/table/{gpt}')
gptTable

,Date,raev100,dev,Riesgo
0,2024-12-30,NaN,0,Muy Alto
1,2025-01-06,NaN,0,Muy Alto
2,2025-01-13,0.0,0,Bajo
3,2025-01-20,1.9,6,Bajo


# Pruebas

In [52]:
name1 = f'{company}_Page_2.csv'
test2 = pd.read_csv(f'./Clientes/{company}/{week}/table/{name1}')
test2

,Date,raev100
0,2024-01-28,9.638533
1,2024-02-04,8.353868
2,2024-02-11,8.363616
3,2024-02-18,7.903602
4,2024-02-25,8.468282
5,2024-03-03,8.836196
6,2024-03-10,7.968901
7,2024-03-17,8.466454
8,2024-03-24,8.547404
9,2024-03-31,8.445357


In [124]:
name2 = f'{company}_Page_3.csv'
test2 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name2)
test2

,Date,ENVASADO,GRANEL,TRANSFERENCIA
0,2024-01-28,10.025124,8.549973,11.724219
1,2024-02-04,9.192174,8.195035,6.322118
2,2024-02-11,9.779429,7.723128,6.672371
3,2024-02-18,8.670188,7.708701,6.361489
4,2024-02-25,10.197999,7.796751,5.978438
5,2024-03-03,9.367366,7.751174,11.155470
6,2024-03-10,9.381377,7.422388,6.552186
7,2024-03-17,9.688577,7.530520,8.755776
8,2024-03-24,10.013832,7.845486,7.276921
9,2024-03-31,10.117122,7.223848,8.011054


In [125]:
name3 = f'{company}_Ranking_3_1.csv'
test3 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name3)
test3

,Flota,raev100,dev
0,TRANSFERENCIA,6.970044,1425.585870
1,GRANEL,7.923689,7356.417954
2,ENVASADO,9.894826,6252.347244


In [126]:
name4 = f'{company}_Ranking_4_1.csv'
test4 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name4)
test4

,Oficina,raev100,dev
0,CURICO,1.660483,9.448623
1,OSORNO,2.868740,192.414009
2,LINARES,3.246898,63.126129
3,CHILOE,4.010607,94.507272
4,VALDIVIA,4.217257,88.236197
5,TALCA,4.251663,570.940631
6,LOS ANGELES,5.066588,309.472975
7,PUERTO MONTT,5.829024,234.824229
8,IQUIQUE,6.214203,121.762423
9,PEÑON,7.033858,813.631891


In [127]:
name5 = f'{company}_Ranking_5_1.csv'
test5 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name5)
test5

,Oficina,21 abr.,28 abr.,dev
0,ANTOFAGASTA,27.057115,29.937423,2681.971000
1,ARICA,7.037737,4.661150,155.608304
2,CHILLAN,10.840296,7.308024,266.556697
3,CHILOE,4.010607,4.482413,198.731286
4,CONCON,8.787822,7.496520,2295.911573
5,COPIAPO,11.323830,10.122546,775.077038
6,COYHAIQUE,14.372453,7.173409,528.526015
7,CURICO,1.660483,2.349780,19.040486
8,IQUIQUE,6.214203,3.254931,161.162607
9,LENGA,8.073020,7.872847,4156.514372


In [128]:
name6 = f'{company}_Ranking_6_1.csv'
test6 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name6)
test6

,Codigo,raev100,dev
0,EN4324,74.073852,11.761089
1,EN5299,55.208432,6.143679
2,EN5846,50.154992,733.577017
3,EN3171,36.859857,22.469758
4,GR1636,36.620591,347.591331
5,EN5142,28.638622,87.605969
6,EN5245,27.646745,5.223260
7,GR14(ES14),26.115406,5.489280
8,EN5158,26.089522,70.402482
9,EN4335,25.863644,180.049295


In [129]:
name7 = f'{company}_Evolucion_7_1.csv'
test7 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name7)
test7

,Date,raev100
0,2024-01-28,10.025124
1,2024-02-04,9.192174
2,2024-02-11,9.779429
3,2024-02-18,8.670188
4,2024-02-25,10.197999
5,2024-03-03,9.367366
6,2024-03-10,9.381377
7,2024-03-17,9.688577
8,2024-03-24,10.013832
9,2024-03-31,10.117122


In [130]:
name8 = f'{company}_Ranking_8_1.csv'
test8 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name8)
test8

,Oficina,raev100,dev
0,CURICO,1.660483,9.448623
1,ARICA,2.789822,11.971521
2,LINARES,3.246898,63.126129
3,OSORNO,3.275433,55.452091
4,CHILOE,4.298033,36.701216
5,IQUIQUE,4.556267,31.589006
6,VALDIVIA,5.347318,53.622913
7,LOS ANGELES,6.258340,204.420862
8,TALCA,7.780536,146.726210
9,PEÑON,7.994359,418.243290


In [131]:
name9 = f'{company}_Ranking_9_1.csv'
test9 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name9)
test9

,Codigo,raev100,dev
0,EN4900,2.343411,0.274934
1,EN3800,4.441727,41.821344
2,EN3807,5.096130,21.611347
3,EN5846,50.154992,733.577017


In [132]:
name10 = f'{company}_Ranking_10_1.csv'
test10 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name10)
test10

,Date,raev100,dev
0,2024-04-07,47.586888,409.302818
1,2024-04-14,49.921333,706.205139
2,2024-04-21,50.154992,733.577017
3,2024-04-28,52.967273,147.662252


In [133]:
name11 = f'{company}_Ranking_11_1.csv'
test11 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name11)
test11

,Date,raev100,dev
0,2024-04-07,5.406657,30.248712
1,2024-04-14,4.034711,13.612399
2,2024-04-21,4.441727,41.821344
3,2024-04-28,6.900500,13.779709


In [134]:
name12 = f'{company}_Ranking_12_1.csv'
test12 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name12)
test12

,Codigo,raev100,dev
0,EN5150,0.000000,0.000000
1,EN5162,8.707411,4.210216
2,EN5145,10.416271,17.488218
3,EN5886,18.101755,208.376676


In [135]:
name13 = f'{company}_Ranking_13_1.csv'
test13 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name13)
test13

,Date,raev100,dev
0,2024-04-07,1.519462,1.933839
1,2024-04-14,4.153521,5.663233
2,2024-04-21,18.101755,208.376676
3,2024-04-28,11.377839,1.185697


In [136]:
name14 = f'{company}_Ranking_14_1.csv'
test14 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name14)
test14

,Date,raev100,dev
0,2024-04-07,5.333717,1.640569
1,2024-04-14,7.357370,41.187894
2,2024-04-21,8.707411,4.210216
3,2024-04-28,7.508165,0.448438


In [137]:
name15 = f'{company}_Ranking_15_1.csv'
test15 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name15)
test15

,Codigo,raev100,dev
0,EN3394,0.000000,0.000000
1,EN4809,7.984042,15.945334
2,EN6277,8.540688,31.550505
3,EN6247,11.265101,23.303338
4,EN5142,28.638622,87.605969


In [138]:
name16 = f'{company}_Ranking_16_1.csv'
test16 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name16)
test16

,Date,raev100,dev
0,2024-04-07,43.414604,70.302372
1,2024-04-14,6.506956,10.281319
2,2024-04-21,7.984042,15.945334
3,2024-04-28,23.126840,17.857089


In [139]:
name17 = f'{company}_Ranking_17_1.csv'
test17 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name17)
test17

,Date,raev100,dev
0,2024-04-07,15.212304,35.942738
1,2024-04-14,4.281993,11.309171
2,2024-04-21,28.638622,87.605969
3,2024-04-28,16.973165,6.058635


In [140]:
name18 = f'{company}_Ranking_18_1.csv'
test18 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name18)
test18

,Date,raev100,dev
0,2024-04-07,7.401440,22.960885
1,2024-04-14,11.005134,33.530315
2,2024-04-21,11.265101,23.303338
3,2024-04-28,1.513255,0.472956


In [141]:
name19 = f'{company}_Evolucion_19_1.csv'
test19 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name19)
test19

,Date,raev100
0,2024-01-28,8.549973
1,2024-02-04,8.195035
2,2024-02-11,7.723128
3,2024-02-18,7.708701
4,2024-02-25,7.796751
5,2024-03-03,7.751174
6,2024-03-10,7.422388
7,2024-03-17,7.530520
8,2024-03-24,7.845486
9,2024-03-31,7.223848


In [142]:
evGranel = filterEv(df, startDate='2024-01-22', endDate='2024-04-28', flotas=['GRANEL'])
evGranel

,Date,raev100
0,2024-01-28,8.549973
1,2024-02-04,8.195035
2,2024-02-11,7.723128
3,2024-02-18,7.708701
4,2024-02-25,7.796751
5,2024-03-03,7.751174
6,2024-03-10,7.422388
7,2024-03-17,7.530520
8,2024-03-24,7.845486
9,2024-03-31,7.223848


In [143]:
name20 = f'{company}_Ranking_20_1.csv'
test20 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name20)
test20

,Oficina,raev100,dev
0,PUERTO MONTT,2.726788,76.419084
1,PEÑON,3.152875,86.289734
2,VALDIVIA,3.217835,34.613284
3,OSORNO,3.239808,80.520402
4,TALCA,3.614578,316.483103
5,LOS ANGELES,3.779331,105.052113
6,CHILOE,3.858267,57.806056
7,SAN FERNANDO,6.180316,379.190359
8,TEMUCO,6.465061,238.968994
9,VILLARRICA,6.750295,120.636588


In [144]:
name21 = f'{company}_Ranking_21_1.csv'
test21 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name21)
test21

,Codigo,raev100,dev
0,GR5095,7.880406,15.434218
1,GR5091,14.453858,139.250396
2,GR1636,36.620591,347.591331


In [145]:
name22 = f'{company}_Ranking_22_1.csv'
test22 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name22)
test22

,Date,raev100,dev
0,2024-04-07,5.590446,14.278146
1,2024-04-14,20.813503,176.892986
2,2024-04-21,7.880406,15.434218
3,2024-04-28,4.580797,0.579740


In [146]:
name23 = f'{company}_Ranking_23_1.csv'
test23 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name23)
test23

,Date,raev100,dev
0,2024-04-07,3.970137,24.908202
1,2024-04-14,20.337303,130.980936
2,2024-04-21,14.453858,139.250396
3,2024-04-28,40.254351,154.117490


In [147]:
name24 = f'{company}_Ranking_24_1.csv'
test24 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name24)
test24

,Date,raev100,dev
0,2024-04-07,23.013159,199.773231
1,2024-04-14,11.279112,75.734803
2,2024-04-21,36.620591,347.591331
3,2024-04-28,3.537658,2.034530


In [148]:
name25 = f'{company}_Ranking_25_1.csv'
test25 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name25)
test25

,Codigo,raev100,dev
0,GR5068,3.430326,27.417880
1,GR22(ES22),12.014942,109.972156
2,GR5079,13.768798,117.924428
3,GR5141,22.492808,92.111297
4,GR14(ES14),26.115406,5.489280


In [149]:
name26 = f'{company}_Ranking_26_1.csv'
test26 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name26)
test26

,Date,raev100,dev
0,2024-04-21,22.492808,92.111297
1,2024-04-28,10.118856,14.754523


In [150]:
name27 = f'{company}_Ranking_27_1.csv'
test27 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name27)
test27

,Codigo,raev100,dev
0,GR5044,0.000000,0.000000
1,GR1626,0.000000,0.000000
2,GR5039,2.116926,15.958474
3,GR5017,3.067307,27.481299
4,GR5054,3.698748,33.134514
5,GR5099,4.515806,33.948957
6,GR5123,4.650786,39.302438
7,GR5028,4.690521,20.329393
8,GR5014,4.873340,38.792310
9,GR5085,5.280213,45.817518


In [151]:
name28 = f'{company}_Ranking_28_1.csv'
test28 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name28)
test28

,Date,raev100,dev
0,2024-04-07,4.937204,11.421675
1,2024-04-14,12.012725,96.113961
2,2024-04-21,4.873340,38.792310
3,2024-04-28,9.352305,12.332072


In [152]:
name29 = f'{company}_Ranking_29_1.csv'
test29 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name29)
test29

,Date,raev100,dev
0,2024-04-07,0.000000,0.000000
1,2024-04-14,9.969388,80.510192
2,2024-04-21,3.067307,27.481299
3,2024-04-28,2.755991,5.206192


In [153]:
name30 = f'{company}_Ranking_30_1.csv'
test30 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name30)
test30

,Date,raev100,dev
0,2024-04-07,8.682724,32.318476
1,2024-04-14,9.948051,60.208604
2,2024-04-21,10.594228,67.369737
3,2024-04-28,33.318078,51.782647


In [154]:
name31 = f'{company}_Evolucion_31_1.csv'
test31 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name31)
test31

,Date,raev100
0,2024-01-21,9.151188
1,2024-01-28,11.724219
2,2024-02-04,6.322118
3,2024-02-11,6.672371
4,2024-02-18,6.361489
5,2024-02-25,5.978438
6,2024-03-03,11.155470
7,2024-03-10,6.552186
8,2024-03-17,8.755776
9,2024-03-24,7.276921


In [155]:
evTransferenia = filterEv(df, startDate='2024-01-15', endDate='2024-04-28', flotas=['TRANSFERENCIA'])
evTransferenia

,Date,raev100
0,2024-01-21,9.151188
1,2024-01-28,11.724219
2,2024-02-04,6.322118
3,2024-02-11,6.672371
4,2024-02-18,6.361489
5,2024-02-25,5.978438
6,2024-03-03,11.155470
7,2024-03-10,6.552186
8,2024-03-17,8.755776
9,2024-03-24,7.276921


In [156]:
evEnvasado = filterEv(df, startDate='2024-01-22', endDate='2024-04-28', flotas=['ENVASADO'])
evEnvasado

,Date,raev100
0,2024-01-28,10.025124
1,2024-02-04,9.192174
2,2024-02-11,9.779429
3,2024-02-18,8.670188
4,2024-02-25,10.197999
5,2024-03-03,9.367366
6,2024-03-10,9.381377
7,2024-03-17,9.688577
8,2024-03-24,10.013832
9,2024-03-31,10.117122


In [157]:
name32 = f'{company}_Ranking_32_1.csv'
test32 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name32)
test32

,Oficina,raev100,dev
0,OSORNO,2.235671,56.441517
1,MAIPU,2.414438,8.067815
2,TALCA,3.902453,107.731318
3,LENGA,7.387323,729.339554
4,PEÑON,8.549354,309.098867
5,TEMUCO,14.559345,152.335435
6,SAN FERNANDO,20.236811,62.571363


In [158]:
name33 = f'{company}_Ranking_33_1.csv'
test33 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name33)
test33

,Codigo,raev100,dev
0,EN4980,4.747869,0.985517
1,EN4548,21.255822,61.585846


In [159]:
name34 = f'{company}_Ranking_34_1.csv'
test34 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name34)
test34

,Date,raev100,dev
0,2024-04-07,7.651045,17.223839
1,2024-04-14,9.647060,30.102558
2,2024-04-21,21.255822,61.585846
3,2024-04-28,15.278926,11.910479


In [160]:
name35 = f'{company}_Ranking_35_1.csv'
test35 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name35)
test35

,Date,raev100,dev
0,2024-04-07,4.020552,1.316323
1,2024-04-14,1.327062,1.749146
2,2024-04-21,4.747869,0.985517
3,2024-04-28,6.265980,7.656964


In [161]:
name36 = f'{company}_Ranking_36_1.csv'
test36 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name36)
test36

,Codigo,raev100,dev
0,FTDR27,14.559345,152.335435


In [162]:
name37 = f'{company}_Ranking_37_1.csv'
test37 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name37)
test37

,Date,raev100,dev
0,2024-04-07,20.011011,142.867603
1,2024-04-14,28.095672,197.787390
2,2024-04-21,14.559345,152.335435
3,2024-04-28,15.777732,0.972160


In [163]:
name38 = f'{company}_Ranking_38_1.csv'
test38 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name38)
test38

,Codigo,raev100,dev
0,EN5994,0.000000,0.000000
1,EN6299,5.348019,76.812640
2,EN6271,10.545633,226.142548
3,EN5299,55.208432,6.143679


In [164]:
name39 = f'{company}_Ranking_39_1.csv'
test39 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name39)
test39

,Date,raev100,dev
0,2024-04-07,9.674545,207.395573
1,2024-04-14,10.561497,235.482401
2,2024-04-21,10.545633,226.142548
3,2024-04-28,9.513233,49.863484


In [165]:
name40 = f'{company}_Ranking_40_1.csv'
test40 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name40)
test40

,Date,raev100,dev
0,2024-04-07,6.105872,38.128772
1,2024-04-14,8.831985,39.923080
2,2024-04-21,0.000000,0.000000
3,2024-04-28,NaN,0.000000


In [166]:
name41 = f'{company}_Ranking_41_1.csv'
test41 = pd.read_csv('Clientes/abastible_consolidado/2024-04-15/table/' + name41)
test41

,Date,raev100,dev
0,2024-04-07,9.748724,188.039241
1,2024-04-14,10.204932,230.023812
2,2024-04-21,5.348019,76.812640
3,2024-04-28,7.784193,3.058620


# Pruebas 2

## Abastible

### Evolución Semanal Riesgo RAEV/100, Total Abastible

In [38]:
evTotal = filterEv(df, '2024-01-22', '2024-04-28')
evTotal

,Date,Total-Raev100,Total-Riesgo
0,2024-01-22,9.6,Alto
1,2024-01-29,8.4,Alto
2,2024-02-05,8.4,Alto
3,2024-02-12,7.9,Medio
4,2024-02-19,8.5,Alto
5,2024-02-26,8.8,Alto
6,2024-03-04,8.0,Medio
7,2024-03-11,8.5,Alto
8,2024-03-18,8.5,Alto
9,2024-03-25,8.4,Alto


### Evolución Semanal Riesgo RAEV/100, Total Flotas

In [176]:
evTotalFlotas = filterEv(df, '2024-01-22', '2024-04-28', ['ENVASADO', 'GRANEL', 'TRANSFERENCIA'])
evTotalFlotas

,Date,Envasado-Raev100,Envasado-Riesgo,Granel-Raev100,Granel-Riesgo,Transferencia-Raev100,Transferencia-Riesgo
0,2024-01-22,10.0,Muy Alto,8.5,Alto,11.7,Muy Alto
1,2024-01-29,9.2,Alto,8.2,Alto,6.3,Medio
2,2024-02-05,9.8,Muy Alto,7.7,Medio,6.7,Medio
3,2024-02-12,8.7,Alto,7.7,Medio,6.4,Medio
4,2024-02-19,10.2,Muy Alto,7.8,Medio,6.0,Bajo
5,2024-02-26,9.4,Alto,7.8,Medio,11.2,Muy Alto
6,2024-03-04,9.4,Alto,7.4,Medio,6.6,Medio
7,2024-03-11,9.7,Alto,7.5,Medio,8.8,Alto
8,2024-03-18,10.0,Muy Alto,7.8,Medio,7.3,Medio
9,2024-03-25,10.1,Muy Alto,7.2,Medio,8.0,Medio


### Ranking Riesgo RAEV/100, Total Flotas Abastible

In [149]:
rankTotalFlotas = filterRanking(df, '2024-04-15', '2024-04-21', ['ENVASADO', 'GRANEL', 'TRANSFERENCIA'])
rankTotalFlotas

,Flota,raev100,dev
2,TRANSFERENCIA,7.0,1426
1,GRANEL,7.9,7356
0,ENVASADO,9.9,6252


### Ranking Riesgo RAEV/100, Total Oficinas Abastible

In [175]:
rankTotalOficinas = filterRanking(df, '2024-04-15', '2024-04-21')
rankTotalOficinas

,Oficina,raev100,dev
7,CURICO,1.7,9
13,OSORNO,2.9,192
10,LINARES,3.2,63
3,CHILOE,4.0,95
20,VALDIVIA,4.2,88
18,TALCA,4.3,571
11,LOS ANGELES,5.1,309
15,PUERTO MONTT,5.8,235
8,IQUIQUE,6.2,122
1,ARICA,7.0,92


### Variación (Ranking) Semanal Riesgo RAEV/100, Oficinas Abastible

In [174]:
rankVarOficinas = filterRanking(df, None, '2024-04-28', ultimas=2)
rankVarOficinas

,Oficina,15 abr.,22 abr.,dev
0,ANTOFAGASTA,27.1,29.9,2682
1,ARICA,7.0,4.7,156
2,CHILLAN,10.8,7.3,266
3,CHILOE,4.0,4.5,199
4,CONCON,8.8,7.5,2296
5,COPIAPO,11.3,10.1,775
6,COYHAIQUE,14.4,7.2,528
7,CURICO,1.7,2.3,19
8,IQUIQUE,6.2,3.3,161
9,LENGA,8.1,7.9,4157


### Ranking Semanal Riesgo RAEV/100, Códigos más riesgosos Abastible

In [151]:
rank60Codigos = filterRanking(df, '2024-04-15', '2024-04-21', top=60, dev=50)
rank60Codigos

,Codigo,raev100,dev
11,GR5025,52.4,97
2,EN5846,50.9,731
21,GR5094,47.7,67
18,GR5082,43.0,144
24,GR5150,42.3,91
8,GR1636,41.1,341
0,EN4335,40.4,75
12,GR5038,39.9,70
15,GR5062,39.3,82
22,GR5129,35.2,56


### Evolución Semanal Riesgo RAEV/100, Flota Envasado

In [177]:
evEnvasado = filterEv(df, '2024-01-22', '2024-04-28', ['ENVASADO'])
evEnvasado

,Date,Envasado-Raev100,Envasado-Riesgo
0,2024-01-22,10.0,Muy Alto
1,2024-01-29,9.2,Alto
2,2024-02-05,9.8,Muy Alto
3,2024-02-12,8.7,Alto
4,2024-02-19,10.2,Muy Alto
5,2024-02-26,9.4,Alto
6,2024-03-04,9.4,Alto
7,2024-03-11,9.7,Alto
8,2024-03-18,10.0,Muy Alto
9,2024-03-25,10.1,Muy Alto


### Ranking Riesgo RAEV/100, Oficinas de la Flota Envasado

In [178]:
rankOficinasEnvasado = filterRanking(df, '2024-04-15', '2024-04-21', ['ENVASADO'])
rankOficinasEnvasado

,Oficina,raev100,dev
7,CURICO,1.7,9
1,ARICA,2.8,12
10,LINARES,3.2,63
13,OSORNO,3.3,55
3,CHILOE,4.3,37
8,IQUIQUE,4.6,32
19,VALDIVIA,5.3,54
11,LOS ANGELES,6.3,204
17,TALCA,7.8,147
14,PEÑON,8.0,418


### Ranking Riesgo RAEV/100, Códigos de la Oficina Antofagasta

In [179]:
rankAntofagastaEnvasado = filterRanking(df, '2024-04-15', '2024-04-21', ['ENVASADO'], 'ANTOFAGASTA')
rankAntofagastaEnvasado

,Codigo,raev100,dev
2,EN4900,2.3,0
0,EN3800,4.4,42
1,EN3807,5.1,22
3,EN5846,50.2,734


### Evolución (Ranking) Riesgo RAEV/100, Código EN5846

In [180]:
rankEN5846 = filterRanking(df, None, '2024-04-22', ['ENVASADO'], 'ANTOFAGASTA', 'EN5846', ultimas=3)
rankEN5846

,Date,raev100,dev
0,2024-04-01,47.6,409
1,2024-04-08,49.9,706
2,2024-04-15,50.2,734
3,2024-04-22,53.0,148


### Evolución (Ranking) Riesgo RAEV/100, Código EN3800

In [181]:
rankEN3800 = filterRanking(df, None, '2024-04-22', ['ENVASADO'], 'ANTOFAGASTA', 'EN3800', ultimas=3)
rankEN3800

,Date,raev100,dev
0,2024-04-01,5.4,30
1,2024-04-08,4.0,14
2,2024-04-15,4.4,42
3,2024-04-22,6.9,14


### Ranking Riesgo RAEV/100, Códigos de la Oficina Coyhaique

In [182]:
rankEnvasadoCoyhaique = filterRanking(df, '2024-04-15', '2024-04-21', ['ENVASADO'], 'COYHAIQUE')
rankEnvasadoCoyhaique

,Codigo,raev100,dev
1,EN5150,0.0,0
2,EN5162,8.7,4
0,EN5145,10.4,17
3,EN5886,18.1,208
